In [ ]:
#Collaborative Filtering

In [ ]:
import pandas as pd
import numpy as np

# Load the MovieLens dataset
ratings_data = pd.read_csv('ratings.csv')
movies_data = pd.read_csv('movies.csv')

# Merge the two datasets on the 'movieId' column
data = pd.merge(ratings_data, movies_data, on='movieId')

# Group the data by 'userId' and create a pivot table with 'movieId' as rows and 'userId' as columns
ratings_matrix = data.pivot_table(index='movieId', columns='userId', values='rating')

# Replace any missing values with zeros
ratings_matrix = ratings_matrix.fillna(0)

# Define a function to calculate the cross-convolution filter for a given movie
def cross_conv_filter(window_size=5):
    # Get the movie id from the user via the keyboard
    movie_id = int(input("Enter a movie id: "))
    
    # Get the ratings for the given movie
    movie_ratings = ratings_matrix.loc[movie_id].values
    
    # Pad the ratings array with zeros on both sides
    padded_ratings = np.pad(movie_ratings, (window_size//2, window_size//2), mode='constant', constant_values=0)
    
    # Define the filters
    h_filter = np.array([-1, 0, 1])
    v_filter = np.array([[-1], [0], [1]])
    
    # Calculate the horizontal convolution
    h_convolved_ratings = np.convolve(padded_ratings, h_filter, mode='valid')
    
    # Pad the horizontal convolved ratings array with zeros on both sides
    padded_h_convolved_ratings = np.pad(h_convolved_ratings, (window_size//2, window_size//2), mode='constant', constant_values=0)
    
    # Calculate the vertical convolution
    v_convolved_ratings = np.convolve(padded_h_convolved_ratings, v_filter.flatten(), mode='valid')
    
    # Get the top 10 highest values
    top_indices = np.argsort(v_convolved_ratings)[-10:]
    
    # Get the movie titles and scores
    movie_indices = ratings_matrix.index[top_indices]
    movie_scores = v_convolved_ratings[top_indices]
    movie_titles = movies_data[movies_data['movieId'].isin(movie_indices)]['title'].values
    
    # Print the recommended movies
    print('Recommended movies for movieId {}:'.format(movie_id))
    for i in range(len(movie_indices)):
        print('{}, score: {:.2f}'.format(movie_titles[i], movie_scores[i]))

# Example usage
cross_conv_filter()


In [ ]:
#Global_max_pooling

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

# Load data
movies = pd.read_csv(
    'movies.dat', 
    sep = '::', 
    header = None, 
    engine = 'python', 
    encoding = 'latin-1',
    names=["movieId", "Title", "Genres"]
)
ratings = pd.read_csv(
    'ratings.dat', 
    sep = '::', 
    header = None, 
    engine = 'python', 
    encoding = 'latin-1',
    names=["userId", "movieId", "rating", "Timestamp"]
)

# Preprocess data
unique_movie_ids = ratings['movieId'].unique()
unique_user_ids = ratings['userId'].unique()

# Create a mapping from movie and user ids to their respective indices in the input tensors
movie_to_idx = {movie_id: i for i, movie_id in enumerate(unique_movie_ids)}
user_to_idx = {user_id: i for i, user_id in enumerate(unique_user_ids)}

# Add the indices to the ratings dataframe
ratings['movie_idx'] = ratings['movieId'].map(movie_to_idx)
ratings['user_idx'] = ratings['userId'].map(user_to_idx)

# Split data into training and test sets
train_df = ratings.sample(frac=0.8, random_state=42)
test_df = ratings.drop(train_df.index)

# Define model architecture
n_users = len(user_to_idx)
n_movies = len(movie_to_idx)
n_factors = 50

# Input layers
movie_input = keras.layers.Input(shape=[1], name='movie')
user_input = keras.layers.Input(shape=[1], name='user')

# Embedding layers
movie_embedding = keras.layers.Embedding(n_movies, n_factors, name='movie_embedding')(movie_input)
user_embedding = keras.layers.Embedding(n_users, n_factors, name='user_embedding')(user_input)

# Reshape embedding layers for compatibility with convolutional layers
movie_embedding_reshaped = keras.layers.Reshape((1, n_factors, 1))(movie_embedding)
user_embedding_reshaped = keras.layers.Reshape((1, n_factors, 1))(user_embedding)

# Convolutional layers
conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(movie_embedding_reshaped)
conv2 = keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(user_embedding_reshaped)
merged = keras.layers.concatenate([conv1, conv2])

# Max pooling layer
pool = keras.layers.MaxPooling2D(pool_size=(1, 2))(merged)

# Flatten layer
flatten = keras.layers.Flatten()(pool)

# Output layer
output = keras.layers.Dense(1, activation='relu')(flatten)

# Define model
model = keras.Model(inputs=[movie_input, user_input], outputs=output)

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model on the training set
history = model.fit(
    [train_df['movie_idx'].values, train_df['user_idx'].values], train_df['rating'].values,
    epochs=10, batch_size=32, verbose=1,
    validation_data=([test_df['movie_idx'].values, test_df['user_idx'].values], test_df['rating'].values),
    callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate([test_df['movie_idx'].values, test_df['user_idx'].values], test_df['rating'].values)
test_rmse = np.sqrt(test_loss)
print("Test RMSE:", test_rmse)
print("Test MSE:", test_loss)


In [ ]:
import matplotlib.pyplot as plt

# Get training and test loss histories
training_loss = history.history['loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'b')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
#Global_average_pooling

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

# Load data
movies = pd.read_csv(
    'movies.dat', 
    sep = '::', 
    header = None, 
    engine = 'python', 
    encoding = 'latin-1',
    names=["movieId", "Title", "Genres"]
)
ratings = pd.read_csv(
    'ratings.dat', 
    sep = '::', 
    header = None, 
    engine = 'python', 
    encoding = 'latin-1',
    names=["userId", "movieId", "rating", "Timestamp"]
)

# Preprocess data
unique_movie_ids = ratings['movieId'].unique()
unique_user_ids = ratings['userId'].unique()

# Create a mapping from movie and user ids to their respective indices in the input tensors
movie_to_idx = {movie_id: i for i, movie_id in enumerate(unique_movie_ids)}
user_to_idx = {user_id: i for i, user_id in enumerate(unique_user_ids)}

# Add the indices to the ratings dataframe
ratings['movie_idx'] = ratings['movieId'].map(movie_to_idx)
ratings['user_idx'] = ratings['userId'].map(user_to_idx)

# Split data into training and test sets
train_df = ratings.sample(frac=0.8, random_state=42)
test_df = ratings.drop(train_df.index)

# Define model architecture
n_users = len(user_to_idx)
n_movies = len(movie_to_idx)
n_factors = 50

# Input layers
movie_input = keras.layers.Input(shape=[1], name='movie')
user_input = keras.layers.Input(shape=[1], name='user')

# Embedding layers
movie_embedding = keras.layers.Embedding(n_movies, n_factors, name='movie_embedding')(movie_input)
user_embedding = keras.layers.Embedding(n_users, n_factors, name='user_embedding')(user_input)

# Reshape embedding layers for compatibility with convolutional layers
movie_embedding_reshaped = keras.layers.Reshape((1, n_factors, 1))(movie_embedding)
user_embedding_reshaped = keras.layers.Reshape((1, n_factors, 1))(user_embedding)

# Convolutional layers
conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(movie_embedding_reshaped)
conv2 = keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(user_embedding_reshaped)
merged = keras.layers.concatenate([conv1, conv2])

# Average pooling layer
pool = keras.layers.AveragePooling2D(pool_size=(1, 2))(merged)

# Flatten layer
flatten = keras.layers.Flatten()(pool)

# Output layer
output = keras.layers.Dense(1, activation='relu')(flatten)

# Define model
model = keras.Model(inputs=[movie_input, user_input], outputs=output)

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model on the training set
history = model.fit(
    [train_df['movie_idx'].values, train_df['user_idx'].values], train_df['rating'].values,
    epochs=10, batch_size=32, verbose=1,
    validation_data=([test_df['movie_idx'].values, test_df['user_idx'].values], test_df['rating'].values),
    callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate([test_df['movie_idx'].values, test_df['user_idx'].values], test_df['rating'].values)
test_rmse = np.sqrt(test_loss)
print("Test RMSE:", test_rmse)
print("Test MSE:", test_loss)


In [ ]:
import matplotlib.pyplot as plt

# Get training and test loss histories
training_loss = history.history['loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();